In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import scipy.optimize as opt
from scipy.optimize import LinearConstraint, Bounds

In [3]:
#reading in orlin's cleaned csv data
local_filepath = "/Users/leonlu/Downloads/gold_data.csv" # TODO: I can't import gold_data.csv as a dataframe
gold_data = pd.read_csv(local_filepath, index_col=0)
display(gold_data.head())

,body,html{height:100%;width:100%}.button{border:none;cursor:pointer;color:#0b57d0;font-family:Google Sans Text,Roboto,sans-serif;font-size:14px;border-radius:100px;padding:12px;margin:0 8px;text-decoration:none}.button.primary-button{background-color:#0b57d0;color:#fff;padding:12px 24px}.icon{height:48px}.container{width:800px;height:212px;-webkit-box-orient:vertical;-webkit-box-direction:normal;-webkit-flex-direction:column;-moz-box-orient:vertical;-moz-box-direction:normal;-ms-flex-direction:column;flex-direction:column;display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex;-webkit-box-align:center;-webkit-align-items:center;-moz-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:justify;-webkit-justify-content:space-between;-moz-box-pack:justify;-ms-flex-pack:justify;justify-content:space-between}.error,.login,.request-storage-access{display:none}.error,.login.1,.request-storage-access,.too-many-login-redirects{height:100%;width:100%;-webkit-box-align:center;-webkit-align-items:center;-moz-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-webkit-justify-content:center;-moz-box-pack:center;-ms-flex-pack:center;justify-content:center}.document-root.show-error .error,.document-root.show-login-page .login,.document-root.show-storage-access .request-storage-access,.too-many-login-redirects{display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex}.title-text{font-family:Google Sans,Roboto.1,sans-serif;font-size:28px;line-height:36px}.subtitle-text{font-family:Google Sans Text,Roboto.2,sans-serif;font-size:16px;line-height:24px}
"<!DOCTYPE html><style nonce=""94egGagWic13rgAsNPwrgQ"">.document-root",,,,,,,,,,,,,,,,
"/*# sourceMappingURL=style.css.map */</style><script nonce=""O98UMmcjg0SVFXbXzwi10w"">'use strict';var e=[""storage_access_granted""",not_in_iframe,login_counter];function f(a,b,"c){c=void 0===c?""true"":c;a=new URL(a);for(var ...",c);return a.toString()};/*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Copyright The Closure Library Authors.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SPDX-License-Identifier: Apache-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
*/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"function h(){var a=document.getElementsByClassName(""document-root"")[0]",b=this;this.g=new l;this.h=a;a.getElementsByCl...,function(){return void m(b)});a.getElementsByC...,function(c){return void n(c)})},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#construct reward matrix R_{i,t} of dimension (K x N)
reward_df = gold_data.iloc[:, -6:].pct_change()
reward_df.dropna(how = 'all', inplace = True)
reward_df.drop(reward_df.index[-1], inplace=True) #drops the last line of data
display(reward_df)

IndexError: index -1 is out of bounds for axis 0 with size 0

### Implementation of Huo and Fu begins here

In [6]:
K = 6 #number of peripheral assets (non AUSHF) in market
gamma_ = 0.95 #confidence level for CVaR performance function F
delta = 5
lambda_ = np.random.uniform(0, 1) # risk preference (->1 means UCB1, ->0 means min CVaR)
time_horizon = 10 #number of time-steps/episodes
times_selected = np.zeros(K)

reward_df = np.random.normal(size=(K, time_horizon))
H_matrix = reward_df[:delta] # temporary definition; change to historical_returns
R_matrix = reward_df

In [7]:
# define helper functions used in huo/fu paper
# NOTE: Leon's modification of I, which I respectfully think is more confusing; moreover, the argmax has a large error...
# def I(t):
#     """
#     Definition: Equation 2.1; follows the UCB1 algorithm.
#     t: the time
#     """
#     # Commented out Leon's definition of \bar{R}; using mean(axis=1) is much more concise though
#     # def R_bar(i, t):
#     #     subset = reward_df.iloc[:t+1, i]
#     #     return subset.mean()
#     if t < K: # strict inequality due to assets being 0, 1, ..., K-1
#         selected_asset = t
#     else:
#         # Commented out Leon's definition of \bar{R}
#         # R_bars = []
#         # for j in range(len(reward_df.columns)):
#         #     R_bars.append(R_bar(j, t))
#         R_bar = reward_df.mean(axis=1)
#         return np.argmax(R_bar) + np.sqrt(2 * np.log(t) / T(i, t-1)) #need to define T
    
def I(t):
    """
    Definition: Equation 2.1
    """
    if t < K: # strict inequality due to assets being 0, 1, ..., K-1
        selection = t
    else:
        R_bar = R_matrix.mean(axis=1) # take the mean across the time axis
        selection = np.argmax([R_bar[i] + np.sqrt((2 * np.log(t) / times_selected[i] * (t - 1))) for i in range(K)])
    times_selected[selection] += 1
    return selection
        
def F(gamma, u, alpha, t, delta):
    """
    Definition: Equation 2.3; estimates the conditional value-at-risk.
    H_matrix: the s-th column of H_matrix (K rows, \delta columns) is the historical returns of the s-th asset
    R_matrix: the s-th column of R_matrix (K rows, t - 1 columns) is the trial of returns observed so far of the s-th asset
    """
    sum1 = np.sum([np.max(-np.dot(H_matrix.iloc[:, s], u) - alpha, 0) for s in range(delta)])
    sum2 = np.sum([np.max(-np.dot(R_matrix.iloc[:, s], u) - alpha, 0) for s in range(t - 1)])
    return alpha + (sum1 + sum2) / ((delta + t - 1) * (1 - gamma))

In [8]:
#computation of different omega components
def omega_C(t):
    """
    Definition: Equation 2.4
    Computes a risk-aware portfolio according to Equation 2.3.
    t: the time
    """
    # the objective function at iteration t
    def F_t(args):
        u, alpha = args[0], args[1]
        return F(gamma_, u, alpha, t, H_matrix, R_matrix)

    # constraining u to be a probability distribution over 1, 2, ..., K
    sum_to_one = LinearConstraint(np.ones(K), [0], [1]) # alpha has not been constrained (actually, it just needs to be real)
    positivity_bounds = Bounds(np.zeros(K), np.ones(K))

    # scipy.optimize.minimize
    x0 = np.ones(K + 1)
    x0[:K] /= K # first K components should sum to 1; no idea what alpha (last component) should represent
    approx_min = opt.minimize(
        F_t, x0=x0,
        constraints=[sum_to_one],
        bounds=positivity_bounds
    )
    return approx_min

In [9]:
def sequential_selection_algo(K, lambda_, gamma=gamma_, time_horizon=time_horizon):
    """
    K: number of assets
    gamma: confidence level
    lambda_: risk preference
    """
    returns = []
    for t in range(time_horizon):
        # Equation. 2.2: compute omega_M(t)
        omega_M = np.zeros(K) 
        omega_M[I(t)] = 1
        
        # Equation 2.4: compute the risk aware portfolio
        risk_aware_portfolio = omega_C(t)

        # Equation 2.5: compute convex combination of UCB1 portfolio and risk aware portfolio
        omega_star = lambda_ * omega_M + (1 - lambda_) * risk_aware_portfolio

        # "Receive portfolio reward"
        returns_by_t = np.dot(omega_star, R_matrix[:, t])
        returns.append(returns_by_t)
    return returns # idk what to return; there might be a formula somewhere

In [10]:
sequential_selection_algo(K, lambda_, gamma=gamma_, time_horizon=time_horizon)

ValueError: shapes (1,6) and (7,) not aligned: 6 (dim 1) != 7 (dim 0)